In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
from dask.distributed import Client
from dask.distributed import progress

In [2]:
# bad_df_shape = (3847460917, 6)
bad_df_shape =   (3200000000, 6)

In [3]:
from dask.distributed import Client
from dask.distributed import progress

# Create a connection to the Dask cluster. The dask-scheduler must be running on
# the same machine as this notebook and listening for inbound clients at port 5555.
client = Client('localhost:5555')

OSError: Timed out trying to connect to 'tcp://localhost:5555' after 10 s: in <distributed.comm.tcp.TCPConnector object at 0x113ae1630>: ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# Collect the descriptions of each worker (<host>:<port>) into a list. Print out how many workers we have.

workers = [x[6:] for x in client.scheduler_info()['workers'].keys()]
len(workers)

In [ ]:
workers[:5]

In [ ]:
rows_per_worker = int((bad_df_shape[0] + len(workers) - 1) / len(workers))
rows_per_worker

In [ ]:
import dask.array as da
import dask

def load_chunk_of_bad_data(nrecords):
    return np.random.rand(nrecords, 6)

chunks_per_worker = 10
chunk_size = int(rows_per_worker / chunks_per_worker)

all_delayed = []
for w in range(len(workers)):
    start_row = w * rows_per_worker
    end_row = (w + 1) * rows_per_worker
    if end_row > bad_df_shape[0]:
        end_row = bad_df_shape[0]

    worker = workers[w]
    x = dask.delayed(load_chunk_of_bad_data)(end_row - start_row)
    x = da.from_delayed(x, shape = (end_row - start_row, 6), dtype=float)
    x = x.rechunk((chunk_size, -1))
    all_delayed.append(x)

In [ ]:
# Concatenate all of the arrays we just loaded individually on each worker into a single, logical, global array
# so that we can operate on it as a single entity.

dist_bad_df = da.concatenate(all_delayed, axis = 0)
dist_bad_df = dist_bad_df.persist()
dist_bad_df

In [ ]:
%%time
# Just to make sure everything worked, glance at the first 5 rows of our globally distributed array.
dist_bad_df[:5, :].compute()

In [ ]:
%%time
# Heck of a lot faster the second time because of lazy evaluation
dist_bad_df[:5, :].compute()

In [ ]:
%%time
# Ask dask to rebalance the data across worker nodes. This may do nothing, but we've also seen dask
# make some very dump load balancing decisions (i.e. all data on one node) so we do this to be safe.
client.rebalance(workers)

In [ ]:
# Verify that the shape reported for our global dask array matches the shape of the array we originally loaded from disk locally

dist_bad_df.shape

In [ ]:
# Necessary for KMeans to accept this array for clustering.

dist_bad_df = dist_bad_df.rechunk((dist_bad_df.chunks[0], -1))

In [ ]:
dist_bad_df.chunks[1]

In [ ]:
from dask_ml.cluster import KMeans
import time
import sys

# Use Dask's distributed KMeans implementation to produce clusters (instead of sklearn's mini batch kmeans).
#
# Here we try the clustering 10 times with a different random seed each time, and then choose the best result.

nrepeats = 5
niters = 7
models = []
elapsed_times = []
for r in range(nrepeats):
    model = KMeans(n_clusters = 2, max_iter = niters, tol = 1e-30)
    
    start_time = time.time()
    transformed = model.fit(dist_bad_df)
    elapsed_time = time.time() - start_time

    models.append(model)
    elapsed_times.append(elapsed_time)
    sys.stderr.write('repeat ' + str(r) + ' took ' + str(elapsed_time) + ' s\n')

elapsed_times

In [ ]:
for m in range(len(models)):
    if model.n_iter_ != niters:
        print(m, model.n_iter_, niters)

In [ ]:
print(sum(elapsed_times) / len(elapsed_times), min(elapsed_times))

In [ ]:
%%time
local_df = dist_bad_df.compute()

In [ ]:
client.close()

In [ ]:
from sklearn.cluster import KMeans

models = []
elapsed_times = []
for r in range(nrepeats):
    model = KMeans(n_clusters = 2, max_iter = niters + 1, tol = 1e-30, n_init = 1)
    
    start_time = time.time()
    transformed = model.fit(local_df)
    elapsed_time = time.time() - start_time
    
    models.append(model)
    elapsed_times.append(elapsed_time)

elapsed_times

In [ ]:
for model in models:
    assert model.n_iter_ == niters, (model.n_iter_, niters)

In [ ]:
sum(elapsed_times) / len(elapsed_times)

In [ ]:
sum(elapsed_times) / len(elapsed_times)

### Looking at dask scaling on hpchw4831, hpchw4832, hpchw4833, hpchw4834

|model   | # records     | clusters | repeats | iterations | machines | processes per machine | threads per process | elapsed time (s)   |
|--------|---------------|----------|---------|------------|----------|-----------------------|---------------------|--------------------|
|sklearn | 100,000,000   | 2        | 5       | 5          | 1        |                       |                     | 57.37087316513062  |
|sklearn | 200,000,000   | 2        | 5       | 5          | 1        |                       |                     | 114.86511063575745 |
|sklearn | 400,000,000   | 2        | 5       | 5          | 1        |                       |                     | 229.99075269699097 |
|sklearn | 800,000,000   | 2        | 5       | 5          | 1        |                       |                     | OOM                |
| |  |         |        |           |         |                       |                     |  |
|dask    | 400,000,000   | 2        | 5       | 5          | 4        | 8                     | 3                   |    |
|dask    | 800,000,000   | 2        | 5       | 5          | 4        | 8                     | 3                   |    |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 4        | 8                     | 3                   |    |
| |  |         |        |           |         |                       |                     |  |
|dask    | 800,000,000 | 2        | 5       | 5          | 2 (hpchw4831,2)               | 8                        | 3                  | 330.03280782699585 |
|dask    | 800,000,000 | 2        | 5       | 5          | 4 (hpchw4831,2,3,4)           | 8                        | 3                  | 255.93871474266052 |
|dask    | 800,000,000 | 2        | 5       | 5          | 6 (hpchw4831,2,3,4,5,6)       | 8                        | 3                  | 249.29323554039001 |
|dask    | 800,000,000 | 2        | 5       | 5          | 8 (hpchw4831,2,3,4,5,6,7,8)   | 8                        | 3                  | 192.52172231674194 |
| |  |         |        |           |         |                       |                     |  |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 2 (hpchw4831,2)               | 8                        | 3                  |  |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 4 (hpchw4831,2,3,4)           | 8                        | 3                  |  |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 6 (hpchw4831,2,3,4,5,6)       | 8                        | 3                  | 1120.9423038959503 |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 8 (hpchw4831,2,3,4,5,6,7,8)   | 8                        | 3                  | 313.1888620853424 |

|model   | # records     | clusters | repeats | iterations | machines | processes per machine | threads per process | elapsed time (s)   |
|--------|---------------|----------|---------|------------|----------|-----------------------|---------------------|--------------------|
|sklearn | 100,000,000   | 2        | 5       | 5          | 1        |                       |                     | 57.37087316513062  |
|sklearn | 200,000,000   | 2        | 5       | 5          | 1        |                       |                     | 114.86511063575745 |
|sklearn | 400,000,000   | 2        | 5       | 5          | 1        |                       |                     | 229.99075269699097 |
|sklearn | 800,000,000   | 2        | 5       | 5          | 1        |                       |                     | OOM                |
| |  |         |        |           |         |                       |                     |  |
|dask    | 100,000,000   | 2        | 5       | 5          | 2        | 2                     | 24                  | 123.81572861671448   |
|dask    | 200,000,000   | 2        | 5       | 5          | 2        | 2                     | 24                  | 258.71336379051206 |
| |  |         |        |           |         |                       |                     |  |
|dask    | 100,000,000   | 2        | 5       | 5          | 4        | 2                     | 24                  | 86.6988452911377   |
|dask    | 200,000,000   | 2        | 5       | 5          | 4        | 2                     | 24                  | 141.2972318649292  |
| |  |         |        |           |         |                       |                     |  |
|dask    | 100,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 72.81405272483826  |
|dask    | 200,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 113.55892267227173 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 218.94874792099    |
|dask    | 800,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 371.98787117004395 |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 6        | 2                     | 24                  | 1161.7001044750214 |

|model   | # records     | clusters | repeats | iterations | machines | processes per machine | threads per process | elapsed time (s)   |
|--------|---------------|----------|---------|------------|----------|-----------------------|---------------------|--------------------|
|dask    | 100,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 72.81405272483826  |
|dask    | 200,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 113.55892267227173 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 218.94874792099    |
|dask    | 800,000,000   | 2        | 5       | 5          | 6        | 2                     | 24                  | 371.98787117004395 |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 6        | 2                     | 24                  | 1161.7001044750214 |
| |  |         |        |           |         |                       |                     |  |
|dask    | 100,000,000   | 2        | 5       | 5          | 6        | 4                     | 12                  | 74.7569028377533   |
|dask    | 200,000,000   | 2        | 5       | 5          | 6        | 4                     | 12                  | 108.41277861595154 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 4                     | 12                  | 155.05894827842712 |
|dask    | 800,000,000   | 2        | 5       | 5          | 6        | 4                     | 12                  | 291.241627073288   |
| |  |         |        |           |         |                       |                     |  |
|dask    | 100,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 108.579838514328   |
|dask    | 200,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 135.68784761428833 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 187.0207820892334  |
|dask    | 800,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 295.3570291996002  |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 6        | 8                     | 6                   | 615.1828716754914  |
| |  |         |        |           |         |                       |                     |  |
|dask    | 100,000,000   | 2        | 5       | 5          | 6        | 12                    | 4                   | 150.54672794342042 |
|dask    | 200,000,000   | 2        | 5       | 5          | 6        | 12                    | 4                   | 177.17852234840393 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 12                    | 4                   | 229.65289607048035 (lots of GC) |
| |  |         |        |           |         |                       |                     |  |
|dask    | 100,000,000   | 2        | 5       | 5          | 6        | 8                     | 1                   | 117.88140172958374 |

|model   | # records     | clusters | repeats | iterations | machines | processes per machine | threads per process | elapsed time (s)   |
|--------|---------------|----------|---------|------------|----------|-----------------------|---------------------|--------------------|
|sklearn | 400,000,000   | 2        | 5       | 5          | 1        |                       |                     | 229.99075269699097 |
|dask    | 400,000,000   | 2        | 5       | 5          | 2        | 8                     | 6                   | 264.53171052932737 |
|dask    | 400,000,000   | 2        | 5       | 5          | 4        | 8                     | 6                   | 192.60448741912842 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 187.0207820892334  |

|model   | # records     | clusters | repeats | iterations | machines | processes per machine | threads per process | elapsed time (s)   |
|--------|---------------|----------|---------|------------|----------|-----------------------|---------------------|--------------------|
|dask    | 100,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 108.579838514328   |
|dask    | 200,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 135.68784761428833 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 187.0207820892334  |
|dask    | 800,000,000   | 2        | 5       | 5          | 6        | 8                     | 6                   | 295.3570291996002  |
|dask    | 1,600,000,000 | 2        | 5       | 5          | 6        | 8                     | 6                   | 615.1828716754914  |

|model   | # records     | clusters | repeats | iterations | machines | processes per machine | threads per process | elapsed time (s)   |
|--------|---------------|----------|---------|------------|----------|-----------------------|---------------------|--------------------|
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 2                     | 12                  | 198.51 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 4                     | 6                   | 185.04 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 8                     | 3                   | 216.27  |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 12                    | 2                   | (lots of GC) 341.79 |
|dask    | 400,000,000   | 2        | 5       | 5          | 6        | 24                    | 1                   | (lots of GC) 369.08 |

In [ ]:
# Look at scaling as you increase the number of nodes

scaling = pd.DataFrame([{'# nodes': 2, 'dask elapsed time': 264.53171052932737, 'sklearn elapsed time': 229.99075269699097},
                        {'# nodes': 4, 'dask elapsed time': 192.60448741912842,  'sklearn elapsed time': 229.99075269699097},
                        {'# nodes': 6, 'dask elapsed time': 187.0207820892334, 'sklearn elapsed time': 229.99075269699097}])
scaling.plot(kind = 'line', figsize = (10, 5), x = '# nodes', y = ['dask elapsed time', 'sklearn elapsed time'])

In [ ]:
scaling

In [ ]:
# Look at scaling as you increase the dataset size

scaling = pd.DataFrame([{'# records': 100000000, 'dask elapsed time': 108.579838514328},
                        {'# records': 200000000, 'dask elapsed time': 135.68784761428833},
                        {'# records': 400000000, 'dask elapsed time': 187.0207820892334},
                        {'# records': 800000000, 'dask elapsed time': 295.3570291996002},
                        {'# records': 1600000000, 'dask elapsed time': 615.1828716754914}])
scaling.plot(kind = 'line', figsize = (10, 5), x = '# records', y = ['dask elapsed time'])

In [ ]:
scaling